## Get GHCND data
This script adapts the module created here: https://github.com/scotthosking/get-station-data. It allows easy access to and processing of GHCN historical daily data from >100,000 weather stations across the globe. 

Try git-cloning this repo and installing these packages, following the directions in the ReadMe file.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Jan 24 2022
Updated on Sep 13 2024

@author: Nora Connor
"""

import os
import pandas as pd
import datetime 
import requests
from io import StringIO

#Make sure you git clone the get-station-data Github repo before you run these lines of code:
from get_station_data import ghcnd
import get_station_data.util as ghcn_utils

#Edit this line to point to your preferred output directory:
output_directory = r'{Add_Your_Output_Directory}'

In [ ]:
def get_weather_data(site,url):
   
    try:
        r = requests.get(url)
        output = r.text
        site_data = pd.read_csv(StringIO(output))
    except:
        print('An exception occurred when attempting to get site ' + str(site))

    return site_data

You'll need to determine which weather stations you want to pull data from. Try using this command:

    stations = ghcnd.get_stn_metadata()

One idea would be to pull the weather station that is closest to the center of each state. You'll need to do your own investigation to find the (Latitude, Longitude) for a state of interest, in decimal format (e.g., (37.820804, -77.107317)). Then, you can try using this command from this same package:

    dists, indices = ghcn_utils.get_nearest_neighbors(stations[['lat', 'lon']], state_centers[['LATDEC', 'LONGDEC']], k=1)

HINT: You can get lat/long decimals by searching for a location on Google Maps, if you need to brute force it.

In [ ]:
#This is where you can set a date range; try experimenting with this block to get different lists of dates:
start_dates = pd.date_range(start=datetime.date(2001,1,1),periods = 20, freq='AS').strftime('%Y-%m-%d').to_list()
end_dates = pd.date_range(start=datetime.date(2001,1,1),periods = 20, freq='A').strftime('%Y-%m-%d').to_list()

#This is where you'll need to figure out what weather stations you want to pull data from.
#The format for stations will look like this: USC00319427, US1NCCA0004, ACW00011604, etc. 
sites = []

In [ ]:
#I didn't clean this for you, so you may have to make some changes to this code to get the weather data that you want. 
#For example, it looks like this code is currently pulling only the precipitation data, and attempting to convert the units into inches.

for site in sites:
    print(site)
    df = pd.DataFrame()

    df['start_dates'] = start_dates
    df['end_dates'] = end_dates
    df['site'] = site
    station = site

    df['url'] = 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&dataTypes=PRCP&stations=' + station + '&startDate=' + df['start_dates'] +'&endDate=' + df['end_dates'] +'&boundingBox=90,-180,-90,180'
    site_data=[]

    for index, row in df.iterrows():

        url = row['url']
        site = row['site']
        tmp = get_weather_data(site, url)
        #print(index)
        if 'Error' in str(tmp):
            print('no data')
        elif 'Empty' in str(tmp):
            print('no data available')
        elif '504 Gateway Time-out' in str(tmp):
            print('504 error')
        else:
            site_data.append(tmp)
    
    try:
        df1 = pd.DataFrame(site_data[0])
        for i in range(len(site_data) -1 ):
            df1 = pd.concat([df1, site_data[i+1]], axis=0)
        df1['PRCP(inches)'] = df1['PRCP']*(1.0/254)
        
        #Output to csv 
        out_data = os.path.join(output_directory, str(site) + '.csv')
        df1.to_csv(out_data)
    
    except:
        print('ERROR: no data at station')